In [0]:

from google.colab import drive
drive.mount('/content/drive')

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
path = "drive/My Drive/logistic.txt"

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from pydrive.auth import GoogleAuth
auth.authenticate_user()
gauth = GoogleAuth()


In [0]:
gauth.credentials = GoogleCredentials.get_application_default()
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
download = drive.CreateFile({'id': '1-Oduujt-fDn-Gkuc29uXSCrf1ejqTp6b'}) #visitors pool for 04_12_18 with choosen features

In [0]:
download.GetContentFile('DOWNLOAD.json')

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
df=pd.read_json('DOWNLOAD.json', lines=True)

In [0]:
download_05 = drive.CreateFile({'id': '1RzwqLZb62wYMnqsRT9o_QA3yd949-g4W'}) #visitors pool for next day 05_12_18 with only browserId and unixTimestamp columns

In [0]:
download_05.GetContentFile('DOWNLOAD.json')

In [0]:
visits=pd.read_json('DOWNLOAD.json', lines=True)

In [0]:
df.describe()

In [0]:
df.columns

In [0]:
visitors=visits['browserId']

In [0]:
visitors #the list of visitors on the next day, First-time visitors are excluded

In [0]:
retention = df['browserId'].isin(visitors) #determine visitors who returns next day with 'True' value
df['Ret']=retention

In [0]:
df

In [0]:
sum(df['Ret']) #true values

In [0]:
len(df['Ret'])-sum(df['Ret']) #false values, we see that data is unbalanced

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import featuretools as ft

In [0]:
df_majority = df[df['Ret']==False] #balancing our dataset
df_minority = df[df['Ret']==True]
# resample minority class

from sklearn.utils import resample
df_majority_upsampled = resample(df_majority, 
                                 replace=True,     
                                 n_samples=43219, random_state=123)   
df_up = pd.concat([df_minority, df_majority_upsampled])
df_up['Ret'].value_counts() #resampled dataframe

In [0]:
X = df_up['contentAuthor'] #one feature as input
Y = df_up['Ret']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
cvec = CountVectorizer(stop_words='english').fit(X_train) #in this work we will use 3 embedding alghorithms, CountVectorizer is the first
df_train=pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())
df_test=pd.DataFrame(cvec.transform(X_test).todense(), columns=cvec.get_feature_names())

In [0]:
lr = LogisticRegression(penalty='l1') #logistic regression for sparse matrice
lr.fit(df_train, Y_train)
lr.score(df_test, Y_test) 

In [0]:
lr = LogisticRegression()
lr.fit(df_train, Y_train)
lr.score(df_test, Y_test) #Spoiler: actually the best score in this job


In [0]:

dfna=df_up.dropna()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']

X5=dfna['contentAuthor']
X6=dfna['contentSection']

X8=dfna['region']
Y=dfna['Ret']
X2=cvec.fit_transform(X2)
X3=cvec.fit_transform(X3)

X5=cvec.fit_transform(X5)
X6=cvec.fit_transform(X6)

X8=cvec.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X5, X6, X8])

In [0]:
(X_train, X_test) = train_test_split(X)
(Y_train, Y_test) = train_test_split(Y)

In [0]:
lr = LogisticRegression(penalty='l1') 
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

In [0]:
from sklearn import tree #decision tree alghorithm
clf = tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')
clf = clf.fit(X_train, Y_train)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
Y_pred=clf.predict(X_test)

In [0]:
accuracy_score(Y_test, Y_pred, normalize=True) #not the best score with multiple categorial features

In [0]:
X = df_up['contentAuthor'] 
Y = df_up['Ret']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train=pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())
df_test=pd.DataFrame(cvec.transform(X_test).todense(), columns=cvec.get_feature_names())
a_train=pd.DataFrame(cvec.transform(X_train).todense(), columns=cvec.get_feature_names())


In [0]:
clf = clf.fit(df_train, Y_train)

In [0]:
Y_pred=clf.predict(df_test) #randomforest classifier for contentAuthor feature

In [0]:
accuracy_score(Y_test, Y_pred, normalize=True) 

In [213]:
from sklearn.feature_extraction.text import TfidfVectorizer
X=dfna['contentAuthor']
X=vectorizer.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

0.5980572433642469

In [0]:
!pip install category_encoders
import category_encoders as ce
ce_binary = ce.BinaryEncoder(cols = ['contentAuthor'])

In [0]:
x=df_up['contentAuthor']
y=df_up['Ret']
x=ce_binary.fit_transform(x) #binary encoder as alternate to TF-Idf, Tf and CVEC encoders

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)

In [0]:
clf = clf.fit(X_train, Y_train)

In [0]:
Y_pred=clf.predict(X_test)

In [0]:
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

In [0]:
x=df_up['contentSection']
y=df_up['Ret']
ce_binary = ce.BinaryEncoder(cols = ['contentSection'])
x=ce_binary.fit_transform(x)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
X=dfna['contentSection']
X=vectorizer.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)


In [212]:
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

0.5986968340262232

In [0]:
x=df_up['contentSection']
y=df_up['Ret']
ce_binary = ce.BinaryEncoder(cols = ['contentSection'])
x=ce_binary.fit_transform(x)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB() #naive Bayes Alghorithm


In [0]:
model.fit(X_train, Y_train)

In [0]:
Y_pred=model.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
dfna=df_up.dropna()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']

X8=dfna['region']
Y=dfna['Ret']

X2=cvec.fit_transform(X2)
X3=cvec.fit_transform(X3)
X4=cvec.fit_transform(X4)
X5=cvec.fit_transform(X5)
X6=cvec.fit_transform(X6)

X8=cvec.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

In [0]:
from sklearn.neighbors import KNeighborsClassifier #knn for multiple features
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(X_train, Y_train) 

In [0]:
Y_pred=neigh.predict(X_test) 
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF for embedding
vectorizer = TfidfVectorizer()
X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']
X8=dfna['region']
Y=dfna['Ret']

X2=vectorizer.fit_transform(X2)
X3=vectorizer.fit_transform(X3)
X4=vectorizer.fit_transform(X4)
X5=vectorizer.fit_transform(X5)
X6=vectorizer.fit_transform(X6)
X8=vectorizer.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [0]:
neigh.fit(X_train, Y_train)
Y_pred=neigh.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

In [202]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

0.5859421560035057

In [0]:
from sklearn.feature_extraction.text import FeatureHasher #feature hashing
hasher = FeatureHasher(input_type='string')

X1=dfna['adblockerStatus']
X2=dfna['browser']
X3=dfna['city']
X4=dfna['contentAuthor']
X5=dfna['contentSection']
X6=dfna['browser']
X8=dfna['region']
Y=dfna['Ret']

X2=hasher.fit_transform(X2)
X3=hasher.fit_transform(X3)
X4=hasher.fit_transform(X4)
X5=hasher.fit_transform(X5)
X6=hasher.fit_transform(X6)
X8=hasher.fit_transform(X8)
from scipy.sparse import coo_matrix, vstack, hstack
X=hstack([X2, X3, X4, X5, X6, X8])

In [205]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33) #for all of  the 5 alghorithms
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

0.5846450482033304

In [206]:
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

0.5448904469763366

In [207]:
neigh.fit(X_train, Y_train)
Y_pred=neigh.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

0.5912007011393514

In [208]:
lr.fit(X_train, Y_train)
Y_pred=lr.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)

0.5933742331288343

In [218]:
X=dfna['hour']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.33)
clf = clf.fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
accuracy_score(Y_test, Y_pred, normalize=True)



0.5779141104294478